In [10]:
# check_brackets
def check_brackets(input_string):
    br_opening = '{[('
    br_closing = '}])'
    br_closes = dict(zip(br_closing, br_opening))
    br_stack = []
    for ch in input_string:
        if ch in br_opening:
            br_stack.append(ch)
        if ch in br_closing:
            if br_stack and br_closes[ch] == br_stack[-1]:
                br_stack.pop()
            else:
                return False
    return br_stack == []
        


check_brackets("[({]})"), check_brackets("([{}({}[])])")

(False, True)

keep a stack of open brackets:
- pop off top if close, 
- fail if close does not match open, 
- fail if not empty at end of string

In [5]:
test = [1, 2, 3]
test.pop()

3

In [9]:
if [1]:
    print('ha')

ha
